## Laboratorio 01
Contruya un mlp para clasificación de imágenes, utilizando el código de los cuadernillos proporcionados para su revisión, el dataset que utilizara será elegido por cada uno y se coordinara con el estudiante Layme Gonzales Marco Antonio, para evitar repetir, se debe realizar una descripción detallada del código desarrollado, como de los resultados obtenidos. (No se debe utilizar Pytorch)

Es obligatorio copiar a esta plataforma el archivo py o ipynb, además de la dirección del repositorio en gitlab.


# Importacion necesaria

*  NumPy: Para manejar matrices y cálculos numéricos.
*  Matplotlib: Para la visualización de datos.
*  Scikit-learn:
    * load_digits(): Carga el dataset de dígitos escritos a mano.
    * train_test_split(): Divide los datos en entrenamiento y prueba.
    * OneHotEncoder(): Convierte las etiquetas en formato one-hot.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml

#Cargar dataset de dígitos

  * Cargamos el dataset Digits, que contiene imágenes de 8x8 píxeles representadas como vectores de 64 valores.
  * Normalizamos los valores de los píxeles dividiéndolos entre 255
  * Ajustamos la forma de las etiquetas para facilitar el procesamiento.

In [ ]:
# Cargar MNIST (28x28 imágenes en escala de grises)
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist.data, mnist.target.astype(np.int32)

# Normalizar imágenes (valores entre 0 y 1)
X = X / 255.0

# Dividir en train/test (60k train, 10k test)
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]


###MLP

In [ ]:
class MLP:
    def __init__(self, input_size, hidden_size, output_size):
        # Inicializar pesos aleatorios
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros(hidden_size)
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros(output_size)

    def forward(self, X):
        # Capa oculta (ReLU)
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = np.maximum(0, self.z1)  # ReLU

        # Capa de salida (Softmax)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        exp_z = np.exp(self.z2 - np.max(self.z2, axis=1, keepdims=True))
        self.probs = exp_z / np.sum(exp_z, axis=1, keepdims=True)
        return self.probs

    def backward(self, X, y, learning_rate):
        m = X.shape[0]  # Número de ejemplos

        # Gradiente de la pérdida respecto a z2
        dZ2 = self.probs
        dZ2[range(m), y] -= 1
        dZ2 /= m

        # Gradientes capa 2
        dW2 = np.dot(self.a1.T, dZ2)
        db2 = np.sum(dZ2, axis=0)

        # Gradiente capa oculta (ReLU)
        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * (self.z1 > 0)  # Gradiente de ReLU

        # Gradientes capa 1
        dW1 = np.dot(X.T, dZ1)
        db1 = np.sum(dZ1, axis=0)

        # Actualizar pesos
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2

    def train(self, X, y, epochs=100, learning_rate=0.1):
        for epoch in range(epochs):
            probs = self.forward(X)
            self.backward(X, y, learning_rate)

            # Calcular pérdida y exactitud
            loss = -np.log(probs[range(len(y)), y]).mean()
            preds = np.argmax(probs, axis=1)
            accuracy = np.mean(preds == y)

            if epoch % 10 == 0:
                print(f"Época {epoch}, Pérdida: {loss:.4f}, Exactitud: {accuracy:.4f}")

    def predict(self, X):
        return np.argmax(self.forward(X), axis=1)

###Entrenar y Evaluar

In [ ]:
mlp = MLP(input_size=784, hidden_size=128, output_size=10)  # MNIST: 784 (28x28), 10 clases
mlp.train(X_train, y_train, epochs=100, learning_rate=0.1)

# Evaluar en test
test_preds = mlp.predict(X_test)
test_accuracy = np.mean(test_preds == y_test)
print(f"Exactitud en test: {test_accuracy:.4f}")

<ipython-input-18-13acc1ad0f84>:52: RuntimeWarning: invalid value encountered in log
  loss = -np.log(probs[range(len(y)), y]).mean()


Época 0, Pérdida: nan, Exactitud: 0.0000
Época 10, Pérdida: nan, Exactitud: 0.0000
Época 20, Pérdida: nan, Exactitud: 0.0000
Época 30, Pérdida: nan, Exactitud: 0.0000
Época 40, Pérdida: nan, Exactitud: 0.0000
Época 50, Pérdida: nan, Exactitud: 0.0000
Época 60, Pérdida: nan, Exactitud: 0.0000
Época 70, Pérdida: nan, Exactitud: 0.0000
Época 80, Pérdida: nan, Exactitud: 0.0000
Época 90, Pérdida: nan, Exactitud: 0.0000
Exactitud en test: 0.7828
